# Jax Flat to ONNX
For envs trained in colab, which uses an earlier version of brax

## Importing Libraries

In [1]:
import os
os.environ["MUJOCO_GL"] = "egl"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import functools
from etils import epath
import numpy as np
import subprocess

import jax
import jax.numpy as jp
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.acme import running_statistics
from mujoco_playground.config import locomotion_params
from mujoco_playground import locomotion

import tf2onnx
import tensorflow as tf
from tensorflow.keras import layers

import onnxruntime as rt
from orbax import checkpoint as ocp


## Env Config

In [2]:
from hector_pg import constants as consts
env_name = "HectorJoystickFlatTerrain"

ROOT_PATH = consts.ROOT_PATH
cpt_path = consts.ROOT_PATH.parent / 'logs' / 'joystick' / 's2_0830_1'

output_path = f"joystick_s2_0830_1.onnx"

In [3]:
ppo_params = locomotion_params.brax_ppo_config(env_name)

def identity_observation_preprocessor(observation, preprocessor_params):
  del preprocessor_params
  return observation

network_factory=functools.partial(
  ppo_networks.make_ppo_networks,
  **ppo_params.network_factory,
  # We need to explicitly call the normalization function here since only the brax
  # PPO train.py script creates it if normalize_observations is True.
  preprocess_observations_fn=running_statistics.normalize,
)

## Load Env, Net dfn, Checkpoints

In [4]:
env_cfg = locomotion.get_default_config(env_name)
env = locomotion.load(env_name, config=env_cfg)

obs_size = env.observation_size
act_size = env.action_size
print(f"Observation Size: {obs_size}")
print(f"Action Size: {act_size}")

ppo_network = network_factory(obs_size, act_size)

l_hip_yaw: -0.79 to 0.79
l_hip_roll: -0.79 to 0.79
l_hip_pitch: -3.14 to 3.14
l_knee: -3.14 to 3.14
l_ankle: -1.40 to 1.40
r_hip_yaw: -0.79 to 0.79
r_hip_roll: -0.79 to 0.79
r_hip_pitch: -3.14 to 3.14
r_knee: -3.14 to 3.14
r_ankle: -1.40 to 1.40
l_shoulder_yaw: -1.40 to 1.40
l_shoulder_pitch: -3.14 to 1.57
l_shoulder_roll: -1.40 to 1.40
l_elbow: -3.00 to 3.00
r_shoulder_yaw: -1.40 to 1.40
r_shoulder_pitch: -3.14 to 1.57
r_shoulder_roll: -1.40 to 1.40
r_elbow: -3.00 to 3.00
Observation Size: {'privileged_state': (482,), 'state': (402,)}
Action Size: 18


In [5]:
checkpoint_root = epath.Path(cpt_path).expanduser().resolve()

checkpoint_dirs = list(checkpoint_root.glob('*'))
checkpoint_dirs = [ckpt for ckpt in checkpoint_dirs if ckpt.is_dir()]
checkpoint_dirs.sort(key=lambda x: int(x.name))

latest_checkpoint = checkpoint_dirs[-1]
print(f'Loading checkpoint: {latest_checkpoint}')
checkpointer = ocp.PyTreeCheckpointer()
params = checkpointer.restore(latest_checkpoint)

normalizer_params = params[1]
policy_params = params[2]

make_inference_fn = ppo_networks.make_inference_fn(ppo_network)
inference_fn = make_inference_fn(params, deterministic=True)

Loading checkpoint: /home/ps/Documents/hector_dev/src/logs/joystick/s2_0830_1/37355520


/home/ps/Documents/hector_dev/.venv/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1269: UserWarning: Sharding info not provided when restoring. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


## Build TF Network

In [6]:
class MLP(tf.keras.Model):
    def __init__(
        self,
        layer_sizes,
        activation=tf.nn.relu,
        kernel_init="lecun_uniform",
        activate_final=False,
        bias=True,
        layer_norm=False,
        mean_std=None,
    ):
        super().__init__()

        self.layer_sizes = layer_sizes
        self.activation = activation
        self.kernel_init = kernel_init
        self.activate_final = activate_final
        self.bias = bias
        self.layer_norm = layer_norm

        if mean_std is not None:
            self.mean = tf.Variable(mean_std[0], trainable=False, dtype=tf.float32)
            self.std = tf.Variable(mean_std[1], trainable=False, dtype=tf.float32)
        else:
            self.mean = None
            self.std = None

        self.mlp_block = tf.keras.Sequential(name="MLP_0")
        for i, size in enumerate(self.layer_sizes):
            dense_layer = layers.Dense(
                size,
                activation=self.activation,
                kernel_initializer=self.kernel_init,
                name=f"hidden_{i}",
                use_bias=self.bias,
            )
            self.mlp_block.add(dense_layer)
            if self.layer_norm:
                self.mlp_block.add(layers.LayerNormalization(name=f"layer_norm_{i}"))
        if not self.activate_final and self.mlp_block.layers:
            if hasattr(self.mlp_block.layers[-1], 'activation') and self.mlp_block.layers[-1].activation is not None:
                self.mlp_block.layers[-1].activation = None

        self.submodules = [self.mlp_block]

    def call(self, inputs):
        if isinstance(inputs, list):
            inputs = inputs[0]
        if self.mean is not None and self.std is not None:
            print(self.mean.shape, self.std.shape)
            inputs = (inputs - self.mean) / self.std
        logits = self.mlp_block(inputs)
        loc, _ = tf.split(logits, 2, axis=-1)
        return tf.tanh(loc)

def make_policy_network(
    param_size,
    mean_std,
    hidden_layer_sizes=[256, 256],
    activation=tf.nn.relu,
    kernel_init="lecun_uniform",
    layer_norm=False,
):
    policy_network = MLP(
        layer_sizes=list(hidden_layer_sizes) + [param_size],
        activation=activation,
        kernel_init=kernel_init,
        layer_norm=layer_norm,
        mean_std=mean_std,
    )
    return policy_network

mean = params[0]['mean']['state']
std = params[0]['std']['state']

# Convert mean/std jax arrays to tf tensors.
mean_std = (tf.convert_to_tensor(mean), tf.convert_to_tensor(std))

tf_policy_network = make_policy_network(
    param_size=act_size * 2,
    mean_std=mean_std,
    hidden_layer_sizes=ppo_params.network_factory.policy_hidden_layer_sizes,
    activation=tf.nn.swish,
)

example_input = tf.zeros((1, obs_size["state"][0]))
example_output = tf_policy_network(example_input)
print(f"Example output shape: {example_output.shape}")

(402,) (402,)
(402,) (402,)
Example output shape: (1, 18)


## Transfering Weights

In [7]:
def transfer_weights(jax_params, tf_model):
    """
    Transfer weights from a JAX parameter dictionary to the TensorFlow model.
    """
    for layer_name, layer_params in jax_params.items():
        try:
            tf_layer = tf_model.get_layer("MLP_0").get_layer(name=layer_name)
        except ValueError:
            print(f"Layer {layer_name} not found in TensorFlow model.")
            continue
        if isinstance(tf_layer, tf.keras.layers.Dense):
            kernel = np.array(layer_params['kernel'])
            bias = np.array(layer_params['bias'])
            print(f"Transferring Dense layer {layer_name}, kernel shape {kernel.shape}, bias shape {bias.shape}")
            tf_layer.set_weights([kernel, bias])
        else:
            print(f"Unhandled layer type in {layer_name}: {type(tf_layer)}")

    print("Weights transferred successfully.")

transfer_weights(params[1]['params'], tf_policy_network)

Transferring Dense layer hidden_0, kernel shape (402, 1024), bias shape (1024,)
Transferring Dense layer hidden_1, kernel shape (1024, 512), bias shape (512,)
Transferring Dense layer hidden_2, kernel shape (512, 256), bias shape (256,)
Transferring Dense layer hidden_3, kernel shape (256, 36), bias shape (36,)
Weights transferred successfully.


## ONNX Conversion and Inference

In [8]:
# Example inputs for the model
test_input = [np.ones((1, obs_size["state"][0]), dtype=np.float32)]
# Define the TensorFlow input signature
spec = [tf.TensorSpec(shape=(1, obs_size["state"][0]), dtype=tf.float32, name="obs")]
tensorflow_pred = tf_policy_network(test_input)[0]
# Build the model by calling it with example data
print(f"Tensorflow prediction: {tensorflow_pred}")
tf_policy_network.output_names = ['continuous_actions']

# Monkey patch version attribute if missing, quick fix
if not hasattr(tf2onnx, 'version'):
    class DummyVersion:
        __version__ = "1.16.1"
        git_version = "dummy_git"

    tf2onnx.version = DummyVersion()

# opset 11 matches isaac lab.
model_proto, _ = tf2onnx.convert.from_keras(tf_policy_network, input_signature=spec, opset=11, output_path=output_path)

# Run inference with ONNX Runtime
output_names = ['continuous_actions']
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)

# Dummy input
onnx_input = {
  'obs': np.ones((1, obs_size["state"][0]), dtype=np.float32)
}
# Prepare inputs for ONNX Runtime
onnx_pred = m.run(output_names, onnx_input)[0][0]

print("ONNX prediction:", onnx_pred)

(402,) (402,)
Tensorflow prediction: [ 0.6793137   0.8586015   0.9995012   0.99693525  0.8508029   0.8811
  0.6025046   0.10058764  0.94045115  0.15524222  0.99545795  0.61960495
 -0.9834577   0.1801741   0.6666665   0.6031356  -0.5119026   0.5095211 ]
(402,) (402,)


I0000 00:00:1756591642.804636   48453 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1756591642.804806   48453 single_machine.cc:376] Starting new session
I0000 00:00:1756591642.902227   48453 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1756591642.902443   48453 single_machine.cc:376] Starting new session


ONNX prediction: [ 0.6793138   0.85860145  0.9995013   0.99693525  0.85080284  0.88109994
  0.6025049   0.10058852  0.9404512   0.15524174  0.9954578   0.6196054
 -0.9834577   0.18017475  0.66666627  0.60313547 -0.51190245  0.5095225 ]


## Rollout and Visualization

In [9]:
env_cfg = locomotion.get_default_config(env_name)

env = locomotion.load(env_name, config=env_cfg)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)

# Reset action scale to match policy
env._config.action_scale = 0.6

l_hip_yaw: -0.79 to 0.79
l_hip_roll: -0.79 to 0.79
l_hip_pitch: -3.14 to 3.14
l_knee: -3.14 to 3.14
l_ankle: -1.40 to 1.40
r_hip_yaw: -0.79 to 0.79
r_hip_roll: -0.79 to 0.79
r_hip_pitch: -3.14 to 3.14
r_knee: -3.14 to 3.14
r_ankle: -1.40 to 1.40
l_shoulder_yaw: -1.40 to 1.40
l_shoulder_pitch: -3.14 to 1.57
l_shoulder_roll: -1.40 to 1.40
l_elbow: -3.00 to 3.00
r_shoulder_yaw: -1.40 to 1.40
r_shoulder_pitch: -3.14 to 1.57
r_shoulder_roll: -1.40 to 1.40
r_elbow: -3.00 to 3.00


In [10]:
# # Rollout with onnx in mujoco playground
x_vel = 1.0  #@param {type: "number"}
y_vel = 0.0  #@param {type: "number"}
yaw_vel = 0.0  #@param {type: "number"}
command = jp.array([x_vel, y_vel, yaw_vel])

gait_freq = 1.0 # Hz
phase_dt = 2 * jp.pi * env.dt * gait_freq
phase = jp.array([0, jp.pi])

actions = []

states = [state := jit_reset(jax.random.PRNGKey(1927))] 
state.info["phase_dt"] = phase_dt
state.info["phase"] = phase
  
for _ in range(env_cfg.episode_length):
  onnx_input = {'obs': np.array(state.obs["state"].reshape(1, -1))}
  action = m.run(output_names, onnx_input)[0][0]
  state = jit_step(state, jp.array(action))
  print(action)
  state.info["command"] = command

  states.append(state)
  # actions.append(state.info["motor_targets"])
  # actions.append(action)
  if state.done:
    print("Unexpected termination.")
    break

/home/ps/Documents/hector_dev/.venv/lib/python3.11/site-packages/jax/_src/interpreters/xla.py:132: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


[ 0.22055691 -0.12031806 -0.19499691  0.3182894  -0.09068053 -0.27626103
  0.24593939  0.33871058 -0.4988069  -0.31440428 -0.27125308 -0.24948391
 -0.03073425  0.45115328  0.1438384  -0.11727498 -0.30948126  0.13573623]
[ 0.13204227 -0.09487602 -0.260933    0.50361985 -0.20950896 -0.18157463
  0.05248386  0.23339087 -0.02890368 -0.2184573  -0.04455075 -0.25966647
  0.00728457  0.2853266   0.09424474 -0.11058046 -0.18914117  0.04936531]
[ 0.04893283 -0.05303007 -0.17097299  0.56151384 -0.2828825  -0.10499718
 -0.03814983  0.3205582   0.3005613  -0.42536432 -0.0320195   0.08141281
  0.12919772  0.17252187 -0.06197105 -0.15888184 -0.09333438  0.12279385]
[ 0.06353596  0.0761179  -0.02534291  0.33616596 -0.18738464 -0.17150061
 -0.08394843  0.06588241  0.683145   -0.45675978 -0.06627987  0.04715319
  0.17548494 -0.00204904 -0.0079418  -0.11013359 -0.02501506  0.03453588]
[ 0.04900067  0.14785898  0.1551379  -0.4366461  -0.08759867 -0.23269296
 -0.01392109 -0.01885737  0.92387104 -0.4782535

In [11]:
# Local ffmpeg
import os
import subprocess

# Add colon separator between paths!
bin_dir = (ROOT_PATH.parent / "visualize" / "ffmpeg" / "ffmpeg-7.0.2-amd64-static")
os.environ["PATH"] = os.pathsep.join([str(bin_dir), os.environ.get("PATH", "")])

print(subprocess.check_output(["ffmpeg", "-version"]).decode())

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [12]:
import mediapy as media

fps = 1.0 / env.dt

frames = env.render(
    states,
    camera="track",
    width=640,
    height=480,
)

media.show_video(frames, fps=fps, loop=False)

100%|██████████| 77/77 [00:00<00:00, 257.34it/s]


This browser does not support the video tag.
